In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import pypsa
from matplotlib import pyplot as plt

In [ ]:
run_folder = "informs"
scenarios_path = Path(
    f"/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/{run_folder}/networks"
)
figures_path = Path(
    f"/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/{run_folder}/figures"
)

In [ ]:
# Read all networks in the scenarios folder store them in dicts with the scenario name as key
networks = {}
for scenario in scenarios_path.iterdir():
    scenario_name = scenario.stem
    networks[scenario_name] = pypsa.Network(scenario)

In [ ]:
# Extract Network.statistics for each scenario
stats = {}
carriers_dict = {}
for network_name, network in networks.items():
    scenario_name = network_name
    stats[scenario_name] = network.statistics()
    carriers_dict[scenario_name] = network.carriers

In [ ]:
# Add alias to the stats dict

# INFORMS
alias_dict = {
    "elec_s75_c4m_ec_lv1.0_REM-2000SEG_E": "ReEDS",
    "elec_s75_c58m_ec_lv1.0_REM-2000SEG_E": "County",
    "elec_s75_c4m_ec_lv1.0_REM-4000SEG_E": "ReEDS_4000",
    "elec_s75_c58m_ec_lv1.0_REM-4000SEG_E": "County_4000",
}


stats_with_alias = {}
for scenario_name, df in stats.items():
    if scenario_name in alias_dict:
        alias_name = alias_dict[scenario_name]
        stats_with_alias[alias_name] = df
    else:
        stats_with_alias[scenario_name] = df

stats = stats_with_alias

# New ordering
new_order = ["ReEDS", "County"]
stats = {key: stats[key] for key in new_order}

In [ ]:
carriers = carriers_dict["elec_s75_c58m_ec_lv1.0_REM-2000SEG_E"].copy()
carriers["legend_name"] = carriers.nice_name
carriers.loc["DC", "legend_name"] = "Transmission"
carriers.loc["DC", "color"] = "#cf1dab"
carriers.loc["battery", "legend_name"] = "Existing BESS"
carriers.set_index("nice_name", inplace=True)
carriers.sort_values(by="co2_emissions", ascending=False, inplace=True)
carriers
# carriers.loc['8hr_PHS', 'color'] ='#90c200'
# carriers.loc['10hr_PHS', 'color'] ='#90c200'
# carriers.loc['12hr_PHS', 'color'] ='#90c200'

In [ ]:
def scenario_comparison(stats, variable, variable_units, carriers_, title, include_link=False):
    colors_ = carriers_["color"]
    # Create subplots
    fig, axes = plt.subplots(
        nrows=len(planning_horizons), ncols=1, figsize=(8, 1.2 * len(planning_horizons)), sharex=True
    )

    # Ensure axes is always iterable (even if there's only one planning horizon)
    if len(planning_horizons) == 1:
        axes = [axes]

    # Loop through each planning horizon
    for ax, horizon in zip(axes, planning_horizons):
        y_positions = np.arange(len(stats))  # One position for each scenario
        for j, (scenario, df) in enumerate(stats.items()):
            df = df.fillna(0)
            bottoms = np.zeros(len(df.columns))  # Initialize the bottom positions for stacking
            if include_link:
                df = df.loc[df.index.get_level_values(0).isin(["Generator", "StorageUnit", "Link"]), variable]
                df = df.loc[~(df.index.get_level_values(1) == "Ac")]
            else:
                df = df.loc[df.index.get_level_values(0).isin(["Generator", "StorageUnit"]), variable]
            df.index = df.index.get_level_values(1)  # Remove the first level of the index
            df = df.reindex(carriers_.index).dropna()
            # Stack the technologies for each scenario
            for i, technology in enumerate(df.index.unique()):
                values = df.loc[technology, horizon]
                match variable_units:
                    case "GW":
                        values = values / (1e3)
                    case "GWh":
                        values = values / (1e3)
                    case "$MM":
                        values = values / (1e6)
                    case "$B":
                        values = values / (1e9)
                    case _:
                        values = values
                ax.barh(
                    y_positions[j],
                    values,
                    left=bottoms[j],
                    color=colors_[technology],
                    label=technology if j == 0 else "",
                )
                bottoms[j] += values

        # Set the title for each subplot
        ax.text(1.01, 0.5, f"{horizon}", transform=ax.transAxes, va="center", rotation="vertical")
        ax.set_yticks(y_positions)  # Positioning scenarios on the y-axis
        ax.set_yticklabels(stats.keys())  # Labeling y-axis with scenario names
        ax.grid(True, axis="x", linestyle="--", alpha=0.5)

    # Set common labels
    plt.xlabel(f"{variable} [{variable_units}]")

    # Reduce space between subplots
    plt.subplots_adjust(hspace=0)

    # Create legend handles and labels from the carriers DataFrame
    carriers_plotted = carriers_.loc[carriers_.index.intersection(df.index.unique())]
    # legend_handles = [plt.Rectangle((0, 0), 1, 1, color=colors_[tech]) for tech in carriers_.index if tech in df.index.unique()]
    legend_handles = [plt.Rectangle((0, 0), 1, 1, color=colors_[tech]) for tech in carriers_plotted.index]
    fig.legend(
        handles=legend_handles,
        labels=carriers_plotted.legend_name.tolist(),
        loc="lower center",
        bbox_to_anchor=(0.5, -0.4),
        ncol=4,
        title="Technologies",
    )

    # Set super title
    fig.suptitle(title, fontsize=12, fontweight="bold")

    # Adjust layout
    plt.tight_layout()
    plt.savefig(figures_path / Path(f"{variable}_comparison.png"), dpi=300, bbox_inches="tight")
    return fig, axes


# Possible Plots
stats[next(iter(stats.keys()))].columns.get_level_values(0).unique()

In [ ]:
# Define your planning horizons and other variables
variable = "Optimal Capacity"
variable_units = "GW"
title = "Comparison of Transmission Resolution for California 8 MMT Target"
planning_horizons = stats[next(iter(stats.keys()))][variable].columns
scenario_comparison(stats, variable, variable_units, carriers, title, include_link=False)

In [ ]:
variable = "Installed Capacity"
variable_units = "GW"
scenario_comparison(stats, variable, variable_units, carriers, title)

In [ ]:
variable = "Supply"
variable_units = "GWh"
scenario_comparison(stats, variable, variable_units, carriers, title, include_link=False)

In [ ]:
variable = "Capital Expenditure"
variable_units = "$B"
scenario_comparison(stats, variable, variable_units, carriers, title, include_link=True)

In [ ]:
variable = "Operational Expenditure"
variable_units = "$B"
scenario_comparison(stats, variable, variable_units, carriers, title, include_link=True)

In [ ]:
# Plotting Opex and Capex
# Clip lower values of Opex to 0 to not include Production Tax Credit Payments
combined_df = pd.DataFrame(columns=["scenario", "horizon", "capex", "opex", "revenue"])
for j, (scenario, df) in enumerate(stats.items()):
    capex = df.loc[df.index.get_level_values(0).isin(["Generator", "StorageUnit"]), "Capital Expenditure"].sum() / 1e9
    opex = (
        df.loc[df.index.get_level_values(0).isin(["Generator", "StorageUnit"]), "Operational Expenditure"]
        .clip(lower=0)
        .sum()
        / 1e9
    )
    revenue = df.loc[df.index.get_level_values(0).isin(["Generator", "StorageUnit"]), "Revenue"].sum() / 1e9
    combined_df = pd.concat(
        [
            combined_df,
            pd.DataFrame(
                {"scenario": scenario, "horizon": opex.index, "capex": capex, "opex": opex, "revenue": revenue}
            ),
        ]
    )

combined_df.reset_index(drop=True, inplace=True)

# Plotting stacked bar plot with different colors for capex and opex, and hatching for scenarios
fig, ax = plt.subplots(figsize=(6, 6))

# Define separate colors for capex and opex
capex_color = "#1f77b4"  # Blue for Capex
opex_color = "#ff7f0e"  # Orange for Opex
revenue_color = "#2ca02c"  # Green for Revenue

# Define hatching patterns for each scenario
hatches = ["...", ""]  # Hatching patterns for different scenarios

# Plot capex and opex stacked with unique colors and scenario-based hatching
bar_width = 3
index = combined_df["horizon"] + (combined_df.index % 2) * (bar_width + 0.05)  # Grouping by scenario

for i, scenario in enumerate(combined_df["scenario"].unique()):
    scenario_df = combined_df[combined_df["scenario"] == scenario]
    ax.bar(
        scenario_df["horizon"] + i * bar_width - bar_width / 2,
        scenario_df["capex"],
        bar_width,
        label=f"{scenario} Capex",
        color=capex_color,
        hatch=hatches[i],
    )
    ax.bar(
        scenario_df["horizon"] + i * bar_width - bar_width / 2,
        scenario_df["opex"],
        bar_width,
        bottom=scenario_df["capex"],
        label=f"{scenario} Opex",
        color=opex_color,
        hatch=hatches[i],
    )
    # ax.bar(scenario_df['horizon'] + i * bar_width - bar_width / 2,
    #        scenario_df['revenue'],
    #        bar_width,
    #        label=f'{scenario} Revenue',
    #        color=revenue_color,
    #        hatch=hatches[i])

# Labels and title
ax.set_xlabel("Investment Year")
ax.set_ylabel("$B/yr")
ax.set_title("Annualized Opex and Capex")
ax.set_xticks([2030, 2040, 2050])
ax.tick_params(axis="x", pad=3)  # Adjust the pad value to reduce or increase the space

ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.savefig(figures_path / Path("Annualized_Opex_Capex.png"), dpi=300, bbox_inches="tight")

In [ ]:
df = stats["County"]["Revenue"]
# df.loc[df.index.get_level_values(0).isin(["Generator", "StorageUnit"])].clip(lower=0)
df

In [ ]:
variable = "Revenue"
variable_units = "$B"
scenario_comparison(stats, variable, variable_units, carriers, title)

# Compare Renewable Profiles

In [ ]:
n = networks["elec_s75_c4m_ec_lv1.0_REM-2000SEG_E"]
gen_cf = n.generators_t.p_max_pu.mean(axis=0)

n.generators["cf"] = gen_cf
n.generators["cf"].hist(bins=100, hue=n.generators.carrier, alpha=0.5)

In [ ]:
n_exp = pypsa.Network(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/informs/Shelock_Runs/elec_s75_c58m_ec_lv1.5_REM-2000SEG_E.nc"
)

In [ ]:
expandable = n_exp.links[n_exp.links.p_nom_extendable]

expandable[["p_nom", "p_nom_opt", "p_nom_max"]]

(expandable.p_nom - expandable.p_nom_opt).sort_values(ascending=False)

In [ ]:
n_exp.links

In [ ]:
n_exp.global_constraints